In [ ]:
import requests
import pandas as pd

In [ ]:
url = "http://103.82.92.37/DM20232/getdata.php?table=transkip_nilai"
headers = {'api-key': "DM20232UBP"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    df_transkip_nilai = pd.DataFrame(data['data'])

    # Menampilkan jumlah data yang diterima
    jumlah_data = len(df_transkip_nilai)
    print("Data yang diterima:", jumlah_data)

    # Menampilkan 10 data pertama sebagai tabel
    display(df_transkip_nilai.head(10))
else:
    print(f"Gagal mengambil data. Status code: {response.status_code} : {response.text}")

In [ ]:
# Hitung IPS per semester per mahasiswa
ips_per_semester = df_transkip_nilai.groupby(['nim', 'semester', 'nama_mk'])['nilai_total'].mean().reset_index()
display(ips_per_semester)

In [ ]:
import requests
import pandas as pd

url = "http://103.82.92.37/DM20232/getdata.php?table=ms_lulusan"
headers = {'api-key': "DM20232UBP"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    df_ms_lulusan = pd.DataFrame(data['data'])

    # Menampilkan jumlah data yang diterima
    jumlah_data = len(df_ms_lulusan)
    print("Data yang diterima:", jumlah_data)

    # Menampilkan 10 data pertama sebagai tabel
    display(df_ms_lulusan.head(10))
else:
    print(f"Gagal mengambil data. Status code: {response.status_code} : {response.text}")

In [ ]:
# Gabungkan dengan dataset ms_lulusan
merged_data = pd.merge(df_ms_lulusan, ips_per_semester, on='nim', how='left')
display(merged_data)

In [ ]:
# Konversi kolom tanggal_lulus dan tgl_masuk ke tipe data datetime
merged_data['tanggal_lulus'] = pd.to_datetime(merged_data['tanggal_lulus'])
merged_data['tgl_masuk'] = pd.to_datetime(merged_data['tgl_masuk'])

# Hitung durasi studi
merged_data['durasi_studi'] = (merged_data['tanggal_lulus'] - merged_data['tgl_masuk']).dt.days
display(merged_data)

In [ ]:
# Identifikasi nilai yang hilang atau tidak konsisten
print(merged_data.isnull().sum())

# Hapus baris dengan nilai yang hilang
merged_data = merged_data.dropna()

# Hapus mahasiswa pindahan (status_masuk == 1)
merged_data = merged_data[merged_data['status_masuk'] == 0]
display(merged_data)

In [ ]:
# Standardisasi format tanggal
merged_data['tanggal_lulus'] = merged_data['tanggal_lulus'].dt.strftime('%Y-%m-%d')

# Standardisasi jenis kelamin (0: Perempuan, 1: Laki-laki)
merged_data['jenis_kelamin'] = merged_data['jenis_kelamin'].map({0: 'Perempuan', 1: 'Laki-laki'})
display(merged_data)

analisis

In [ ]:
# Identifikasi nilai-nilai yang hilang
missing_values = merged_data['nilai_total'].isnull().sum()
print("Jumlah nilai yang hilang:", missing_values)

# Penanganan nilai yang hilang
# Misalnya, menggantikan nilai yang hilang dengan rata-rata dari kolom 'nilai_total'
mean_nilai_total = merged_data['nilai_total'].mean()
merged_data['nilai_total'] = merged_data['nilai_total'].fillna(mean_nilai_total)

# Periksa kembali kriteria pemilihan data
# Pastikan ada mahasiswa yang lulus tepat waktu dalam dataset
if merged_data[merged_data['status_masuk'] == 0].empty:
    print("Tidak ada mahasiswa yang lulus tepat waktu dalam dataset.")
else:
    # Lakukan perhitungan rata-rata IPS lagi setelah menangani nilai yang hilang
    ips_tepat_waktu = merged_data.loc[merged_data['status_masuk'] == 0, 'nilai_total'].mean()
    print("Rata-rata IPS mahasiswa yang lulus tepat waktu setelah penanganan nilai yang hilang:", ips_tepat_waktu)

In [ ]:
import matplotlib.pyplot as plt

# Hitung jumlah mahasiswa dengan predikat 'Pujian' yang lulus tepat waktu
pujian_tepat_waktu = merged_data.loc[(merged_data['status_masuk'] == 0) & (merged_data['predikat'] == 'Pujian'), 'nim'].count()

# Hitung total mahasiswa yang lulus tepat waktu
total_tepat_waktu = merged_data.loc[merged_data['status_masuk'] == 0, 'nim'].count()

# Hitung proporsi mahasiswa dengan predikat 'Pujian' yang lulus tepat waktu
proporsi_pujian_tepat_waktu = pujian_tepat_waktu / total_tepat_waktu

# Buat visualisasi menggunakan grafik pie
labels = ['Pujian', 'Tidak Pujian']
sizes = [proporsi_pujian_tepat_waktu, 1 - proporsi_pujian_tepat_waktu]
colors = ['gold', 'lightcoral']
explode = (0.1, 0)  # Pisahkan potongan 'Pujian'

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Membuat pie chart menjadi lingkaran
plt.title('Proporsi Mahasiswa dengan Predikat "Pujian" yang Lulus Tepat Waktu')
plt.show()

In [ ]:
# Lakukan one-hot encoding pada kolom 'predikat'
merged_data = pd.get_dummies(merged_data, columns=['predikat'])

# Hitung korelasi antara durasi studi dan predikat kelulusan
korelasi_durasi_predikat = merged_data[['durasi_studi', 'predikat_Pujian']].corr()
print("Korelasi antara durasi studi dan predikat kelulusan:")
print(korelasi_durasi_predikat)

In [ ]:
# Ambil data durasi studi dan predikat_Pujian
durasi_studi = merged_data['durasi_studi']
predikat_pujian = merged_data['predikat_Pujian']

# Buat scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(durasi_studi, predikat_pujian, alpha=0.5)
plt.title('Hubungan antara Durasi Studi dan Proporsi Mahasiswa dengan Predikat "Pujian" yang Lulus Tepat Waktu')
plt.xlabel('Durasi Studi (hari)')
plt.ylabel('Predikat "Pujian" (1: Ya, 0: Tidak)')
plt.grid(True)
plt.show()

In [ ]:
# 4. Perbedaan prestasi akademik berdasarkan jenis kelamin
nilai_total_jenis_kelamin = merged_data.groupby('jenis_kelamin')['nilai_total'].mean()
print("Rata-rata nilai total berdasarkan jenis kelamin:")
print(nilai_total_jenis_kelamin)